### Expert Knowledge Worker

#### this is esseantially a question answering agent that is an expert knowledge worker

#### to be used by comployees of `InsureLLM`, an insurance Tech Startup.

#### the agent needs to be accurate and solution should be low cost.


I will be using RAG(Retrieval Augmented Generation) to ensure the question/answering assistant has high accuract

This is merely a brute-force type of RAG

In [1]:
## imports
import os
from dotenv import load_dotenv
from openai import OpenAI
import glob
import gradio as gr

In [2]:
MODEL = "gpt-4o-mini"

In [3]:
load_dotenv()
os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")
openai_client = OpenAI()

In [10]:
context = {}

employees = glob.glob("knowledge-base/employees/*")

for employee in employees:
    name = employee.split(' ')[-1][:-3]
    doc =""
    with open(employee, "r") as f:
        doc = f.read()
    context[name]=doc

In [18]:
context.keys()

dict_keys(['Chen', 'Spencer', 'Tran', 'Blake', 'Lancaster', 'Thompson', 'Greene', 'Thomson', 'Trenton', 'Harper', 'Bishop', 'Carter'])

In [19]:
print(context['Chen'])

# HR Record

# Alex Chen

## Summary
- **Date of Birth:** March 15, 1990  
- **Job Title:** Backend Software Engineer  
- **Location:** San Francisco, California  

## Insurellm Career Progression
- **April 2020:** Joined Insurellm as a Junior Backend Developer. Focused on building APIs to enhance customer data security.
- **October 2021:** Promoted to Backend Software Engineer. Took on leadership for a key project developing a microservices architecture to support the company's growing platform.
- **March 2023:** Awarded the title of Senior Backend Software Engineer due to exemplary performance in scaling backend services, reducing downtime by 30% over six months.

## Annual Performance History
- **2020:**  
  - Completed onboarding successfully.  
  - Met expectations in delivering project milestones.  
  - Received positive feedback from the team leads.

- **2021:**  
  - Achieved a 95% success rate in project delivery timelines.  
  - Awarded "Rising Star" at the annual company gal

In [20]:
products = glob.glob("knowledge-base/products/*")

for product in products:
    name = product.split('/')[-1][:-3]
    doc =""
    with open(product, "r") as f:
        doc = f.read()
    context[name]=doc

In [22]:
print(context.keys())

dict_keys(['Chen', 'Spencer', 'Tran', 'Blake', 'Lancaster', 'Thompson', 'Greene', 'Thomson', 'Trenton', 'Harper', 'Bishop', 'Carter', 'Rellm', 'Markellm', 'Homellm', 'Carllm'])


In [25]:
SYSTEM_PROMPT = """
You are an expert in answering accurate questions about Insurellm, the Insurance Tech Company.
Give brief, accurate answers.
If you dont know the answer, say so.
Do not make anything up if you have not been provided with relevant context.
"""

In [37]:
def get_relevant_context(message):
    relevant_context=[]
    for context_title, context_details in context.items():
        if context_title.lower() in message.lower():
            relevant_context.append(context_details)
    return relevant_context

In [40]:
get_relevant_context("chen")

['# HR Record\n\n# Alex Chen\n\n## Summary\n- **Date of Birth:** March 15, 1990  \n- **Job Title:** Backend Software Engineer  \n- **Location:** San Francisco, California  \n\n## Insurellm Career Progression\n- **April 2020:** Joined Insurellm as a Junior Backend Developer. Focused on building APIs to enhance customer data security.\n- **October 2021:** Promoted to Backend Software Engineer. Took on leadership for a key project developing a microservices architecture to support the company\'s growing platform.\n- **March 2023:** Awarded the title of Senior Backend Software Engineer due to exemplary performance in scaling backend services, reducing downtime by 30% over six months.\n\n## Annual Performance History\n- **2020:**  \n  - Completed onboarding successfully.  \n  - Met expectations in delivering project milestones.  \n  - Received positive feedback from the team leads.\n\n- **2021:**  \n  - Achieved a 95% success rate in project delivery timelines.  \n  - Awarded "Rising Star" 

In [45]:
def add_context(message):
    relevant_context = get_relevant_context(message)
    if relevant_context:
        message+="\n\n the following additional context might be relevant in answering this question: \n\n"
        for relevant in relevant_context:
            message+=relevant+"\n\n"
    return message


In [46]:
print(add_context("Who is Avery Lancaster"))

Who is Avery Lancaster

 the following additional context might be relevant in answering this question: 

# Avery Lancaster

## Summary
- **Date of Birth**: March 15, 1985  
- **Job Title**: Co-Founder & Chief Executive Officer (CEO)  
- **Location**: San Francisco, California  

## Insurellm Career Progression
- **2015 - Present**: Co-Founder & CEO  
  Avery Lancaster co-founded Insurellm in 2015 and has since guided the company to its current position as a leading Insurance Tech provider. Avery is known for her innovative leadership strategies and risk management expertise that have catapulted the company into the mainstream insurance market.  

- **2013 - 2015**: Senior Product Manager at Innovate Insurance Solutions  
  Before launching Insurellm, Avery was a leading Senior Product Manager at Innovate Insurance Solutions, where she developed groundbreaking insurance products aimed at the tech sector.  

- **2010 - 2013**: Business Analyst at Edge Analytics  
  Prior to joining Inno

In [48]:
def chat(message, history):
    messages = [{"role":"system","content":SYSTEM_PROMPT}]
    for user, assistant in history:
        messages.append({"role":"user", "content":user})
        messages.append({"role":"assistant", "content":assistant})
    message = add_context(message)
    messages.append({"role":"user", "content":message})

    stream = openai_client.chat.completions.create(
        model=MODEL,
        messages=messages,
        stream=True
    )

    response = ""
    for chunk in stream:
        response+=chunk.choices[0].delta.content or ""
        yield response

In [50]:
view = gr.ChatInterface(chat).launch()

/Users/protikmostafa/miniconda3/envs/llms/lib/python3.11/site-packages/gradio/chat_interface.py:345: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
